<a href="https://colab.research.google.com/github/yuvraj108c/4k-video-upscaler-colab/blob/main/4k_Video_Upscaler_Colab_(Real_ESRGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4k Video Upscaler Colab (Real-ESRGAN)

Adapted from: [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN)

Made with ❤️ by: [yuvraj108c](https://github.com/yuvraj108c)

Github repository: https://github.com/yuvraj108c/4k-video-upscaler-colab

# 1. Setup (~1 minute)

In [ ]:
!nvidia-smi

from PIL import Image
import cv2, os, subprocess
from google.colab import drive
from tqdm import tqdm

!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

!pip install -q basicsr facexlib gfpgan ffmpeg ffmpeg-python
!pip install -q -r requirements.txt
!python setup.py develop

mount_drive=False #@param{type:"boolean"}

if mount_drive:
  drive.mount('/content/gdrive/')

# 2. Upscale video

- The upscaled video will be saved to `output_dir`
- If google drive is mounted, it will be also saved at `MyDrive/Upscaled Videos (REAL-ESRGAN)`


In [ ]:
video_path="/content/desert 2.3.mp4" #@param{type:"string"}
output_dir="/content/" #@param{type:"string"}
resolution = "FHD" # @param ["FHD", "2k", "4k"] {type:"string"}
model = "RealESRGAN_x4plus" #@param ["RealESRGAN_x4plus" , "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]
crop_to_fit=True #@param{type:"boolean"}

if not os.path.exists(video_path):
   assert False, "Video file does not exist"

video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

final_width = None
final_height = None
aspect_ratio = float(video_width/video_height)

# Get output resolutions
match resolution:
  case "FHD":
    final_width=1920
    final_height=1080
  case "2k":
    final_width=2560
    final_height=1440
  case "4k":
    final_width=3840
    final_height=2160

if aspect_ratio < 1:
  temp = final_width
  final_width = final_height
  final_height = temp

scale_factor = max(final_width/video_width, final_height/video_height)
isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

while isEven == False:
  scale_factor += 0.01
  isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

if not crop_to_fit:
  print(f"Upscaling from {video_width}x{video_height} to {int(video_width * scale_factor)}x{int(video_height * scale_factor)}, crop_to_fit={crop_to_fit}")
  print(f"Enable crop_to_fit to make output resolution {final_width}x{final_height}")
else:
  print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, crop_to_fit={crop_to_fit}")

!python inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale {scale_factor}

video_name_with_ext = os.path.basename(video_path)
video_name = video_name_with_ext.replace(".mp4","")
upscaled_video_path = f"{output_dir}{video_name}_out.mp4"

final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"

if not crop_to_fit:
  final_video_name = f"{video_name}_upscaled_{int(video_width * scale_factor)}_{int(video_height * scale_factor)}.mp4"

final_video_path = os.path.join(output_dir, final_video_name)

# crop to fit
if crop_to_fit:
  command = f"ffmpeg -loglevel error -hwaccel cuda -y -i '{upscaled_video_path}' -c:v h264_nvenc -filter:v  'crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
  subprocess.run(command,shell=True)
  print("Cropping to fit done..")

command = f"mv '{upscaled_video_path}' '{final_video_path}'"
subprocess.run(command,shell=True)

print(f"Upscaled video saved to: {final_video_path}")

# save to drive
if mount_drive:
  drive_folder = "MyDrive/Upscaled Videos (REAL-ESRGAN)"
  save_directory_drive = f"/content/gdrive/{drive_folder}"
  os.makedirs(save_directory_drive, exist_ok=True)

  command = f"'{final_video_path}' '{save_directory_drive}'/'{final_video_name}'"
  subprocess.run(command,shell=True)
  print(f"Saved to drive: /{drive_folder}/{final_video_name}" )

# 3. Disconnect runtime

In [ ]:
from google.colab import runtime

disconnect_when_finish = True  #@param{type:"boolean"}

if disconnect_when_finish:
  runtime.unassign()